# Querying Lasair to construct Databank for algorithms

In [ ]:
# All imports
import os, io, requests, json
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import lasair

In [2]:
# Input Lasair token
TOKEN = os.getenv("LASAIR_TOKEN", "ebc9dee5598ea21658c352a5e71ca8a33875fc96")  
L = lasair.lasair_client(TOKEN)
print("Lasair client ready")

Lasair client ready


## Querying SN Ia objects

In [3]:
# Lasair columns to pull
obj_cols = [
    "gmag","rmag","g_minus_r","jd_g_minus_r","jdmin",
    "ramean","decmean","glatmean",
    "dmdt_g","mag_g02","mag_g08","mag_g28",
    "maggmax","maggmean","maggmin",
    "mag_r02","mag_r08","mag_r28",
    "dmdt_r","magrmax","magrmin",
    "distpsnr1","dmdt_g_err","dmdt_r_err","sgmag1","srmag1"
]

sh_cols = [
    "classification","catalogue_object_id","separationArcsec",
    "northSeparationArcsec","eastSeparationArcsec","physical_separation_kpc",
    "distance","classificationReliability","major_axis_arcsec"
]

tns_cols = [
    "tns_name","type","ra","decl"
]

# Build query
selected = (
    "objects.objectId AS objectId, "
    + ", ".join(f"objects.{c}" for c in obj_cols) + ", "
    + ", ".join(f"sherlock_classifications.{c}" for c in sh_cols) + ", "
    + "sherlock_classifications.z AS sherlock_z, "
    + ", ".join(f"crossmatch_tns.{c}" for c in tns_cols) + ", "
    + "crossmatch_tns.z AS tns_z, "
    "IF(ABS(objects.glatmean)<15., 'yes', 'no') AS galactic_plane, "
    "(objects.jd_g_minus_r - objects.jdmin) AS g_r_days_from_fl"
)

tables = "objects, sherlock_classifications, watchlist_hits, crossmatch_tns"

conditions = (
    "objects.objectId = sherlock_classifications.objectId "
    "AND objects.objectId = watchlist_hits.objectId "
    "AND watchlist_hits.wl_id = 1 "
    "AND watchlist_hits.name = crossmatch_tns.tns_name "
    "AND crossmatch_tns.type LIKE 'SN Ia%' "
    "AND objects.g_minus_r IS NOT NULL"
)

def fetch_page(limit=1000, offset=0):
    rows = L.query(
        selected=selected,
        tables=tables,
        conditions=conditions,
        limit=limit,
        offset=offset,
    )
    return pd.DataFrame(rows)


# Page through results
BATCH = 10_000
offset = 0
dfs = []
seen = set()

while True:
    df_page = fetch_page(limit=BATCH, offset=offset)
    if df_page.empty:
        break

    # Deduplicate by objectId across pages
    new_mask = ~df_page["objectId"].isin(seen)
    df_new = df_page.loc[new_mask].copy()

    if not df_new.empty:
        dfs.append(df_new)
        seen.update(df_new["objectId"].tolist())

    if len(df_page) < BATCH:
        break

    offset += BATCH


# Post-process and save
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    # Ensure numeric z columns 
    for c in ["major_axis_arcsec"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["major_axis_arcsec"] = df_all["major_axis_arcsec"].fillna(-999)
    
    # Ensure numeric z columns
    for c in ["tns_z", "sherlock_z"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["z_spec"] = df_all["tns_z"].fillna(-999)

    # Keep Sherlock redshift separately
    df_all["z_host"] = df_all["sherlock_z"]

    # "Best available" redshift
    df_all["z_best"] = df_all["tns_z"].combine_first(df_all["sherlock_z"]).fillna(-999)

    # Prefer highest Sherlock reliability per object, then dedupe
    if "classificationReliability" in df_all.columns:
        df_all = (df_all.sort_values("classificationReliability", ascending=False)
                        .drop_duplicates(subset=["objectId"]))
    else:
        df_all = df_all.drop_duplicates(subset=["objectId"])

    # Quick sanity prints
    print("Total unique objects:", len(df_all))
    print("Missing TNS z (tns_z NaN):", int(df_all["tns_z"].isna().sum()))
    print("z_spec == -999:", int((df_all["z_spec"] == -999).sum()))
    print("Missing Sherlock z (sherlock_z NaN):", int(df_all["sherlock_z"].isna().sum()))

else:
    df_all = pd.DataFrame(columns=["objectId"])
    print("Total unique objects: 0")

out_path = "lasair_snia_objects_sherlock_tns.csv"
df_all.to_csv(out_path, index=False)
print(f"Saved: {out_path}")
df_all["z"] = df_all["z_spec"]
df_all.to_csv(out_path, index=False)

Total unique objects: 7783
Missing TNS z (tns_z NaN): 20
z_spec == -999: 20
Missing Sherlock z (sherlock_z NaN): 1714
Saved: lasair_snia_objects_sherlock_tns.csv


In [4]:
in_path  = "lasair_snia_objects_sherlock_tns.csv"          
out_path = "lasair_snia_objects_sherlock_tns_complete.csv"  

# Read CSV and normalize common null-like values to NaN
df = pd.read_csv(
    in_path,
    keep_default_na=True,
    na_values=["", " ", "NA", "N/A", "na", "n/a", "Null", "NULL", "null", "None", "none", "-"]
)

# Strip whitespace from column names
df.columns = df.columns.str.strip()
df = df.drop(columns=["z_best", "z_host", "z_spec", "tns_z", "sherlock_z"])

# Keep track of NaNs
nan_counts = df.isna().sum().sort_values(ascending=False)
total_rows = len(df)
nan_percent = (nan_counts / total_rows * 100).round(2)

nan_report = (
    pd.DataFrame({
        "column": nan_counts.index,
        "nan_count": nan_counts.values,
        "nan_percent": nan_percent.values
    })
    .query("nan_count > 0")  # only columns with at least one NaN
    .reset_index(drop=True)
)

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {df.isna().any(axis=1).sum()}")
print("\nColumns ranked by NaN count (descending):")
if nan_report.empty:
    print("No NaNs found in any column.")
else:
    # nice aligned print
    with pd.option_context("display.max_rows", None, "display.max_colwidth", 100):
        print(nan_report.to_string(index=False))

# Produce no-NaN version (rows with any NaN dropped)
df_clean = df.dropna(how="any").copy()

# Remove duplicate objectIds just in case
if "objectId" in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=["objectId"])

# Save cleaned data
df_clean.to_csv(out_path, index=False)

print(f"\nInput rows:  {len(df)}")
print(f"Kept rows:   {len(df_clean)} (no NaNs in any column)")
print(f"Wrote file:  {out_path}")


Total rows: 7783
Rows with at least one NaN: 3721

Columns ranked by NaN count (descending):
                   column  nan_count  nan_percent
               dmdt_g_err       1863        23.94
                 distance       1711        21.98
               dmdt_r_err       1710        21.97
  physical_separation_kpc       1692        21.74
                   sgmag1        455         5.85
                   srmag1        455         5.85
                distpsnr1        455         5.85
                   dmdt_g        443         5.69
                   dmdt_r        247         3.17
         separationArcsec        119         1.53
classificationReliability        118         1.52
     eastSeparationArcsec        118         1.52
    northSeparationArcsec        118         1.52
      catalogue_object_id        118         1.52

Input rows:  7783
Kept rows:   4062 (no NaNs in any column)
Wrote file:  lasair_snia_objects_sherlock_tns_complete.csv


## Querying SN II objects

In [5]:
# Lasair columns to pull
obj_cols = [
    "gmag","rmag","g_minus_r","jd_g_minus_r","jdmin",
    "ramean","decmean","glatmean",
    "dmdt_g","mag_g02","mag_g08","mag_g28",
    "maggmax","maggmean","maggmin",
    "mag_r02","mag_r08","mag_r28",
    "dmdt_r","magrmax","magrmin",
    "distpsnr1","dmdt_g_err","dmdt_r_err","sgmag1","srmag1"
]

sh_cols = [
    "classification","catalogue_object_id","separationArcsec",
    "northSeparationArcsec","eastSeparationArcsec","physical_separation_kpc",
    "distance","classificationReliability","major_axis_arcsec"
]

tns_cols = [
    "tns_name","type","ra","decl"
]

# Build query
selected = (
    "objects.objectId AS objectId, "
    + ", ".join(f"objects.{c}" for c in obj_cols) + ", "
    + ", ".join(f"sherlock_classifications.{c}" for c in sh_cols) + ", "
    + "sherlock_classifications.z AS sherlock_z, "
    + ", ".join(f"crossmatch_tns.{c}" for c in tns_cols) + ", "
    + "crossmatch_tns.z AS tns_z, "
    "IF(ABS(objects.glatmean)<15., 'yes', 'no') AS galactic_plane, "
    "(objects.jd_g_minus_r - objects.jdmin) AS g_r_days_from_fl"
)

tables = "objects, sherlock_classifications, watchlist_hits, crossmatch_tns"

conditions = (
    "objects.objectId = sherlock_classifications.objectId "
    "AND objects.objectId = watchlist_hits.objectId "
    "AND watchlist_hits.wl_id = 1 "
    "AND watchlist_hits.name = crossmatch_tns.tns_name "
    "AND crossmatch_tns.type LIKE 'SN II%' "
    "AND objects.g_minus_r IS NOT NULL"
)

def fetch_page(limit=1000, offset=0):
    rows = L.query(
        selected=selected,
        tables=tables,
        conditions=conditions,
        limit=limit,
        offset=offset,
    )
    return pd.DataFrame(rows)


# Page through results
BATCH = 10_000
offset = 0
dfs = []
seen = set()

while True:
    df_page = fetch_page(limit=BATCH, offset=offset)
    if df_page.empty:
        break

    # Deduplicate by objectId across pages
    new_mask = ~df_page["objectId"].isin(seen)
    df_new = df_page.loc[new_mask].copy()

    if not df_new.empty:
        dfs.append(df_new)
        seen.update(df_new["objectId"].tolist())

    if len(df_page) < BATCH:
        break

    offset += BATCH


# Post-process and save
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    # Ensure numeric z columns 
    for c in ["major_axis_arcsec"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["major_axis_arcsec"] = df_all["major_axis_arcsec"].fillna(-999)
    
    # Ensure numeric z columns
    for c in ["tns_z", "sherlock_z"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["z_spec"] = df_all["tns_z"].fillna(-999)

    # Keep Sherlock redshift separately
    df_all["z_host"] = df_all["sherlock_z"]

    # "Best available" redshift
    df_all["z_best"] = df_all["tns_z"].combine_first(df_all["sherlock_z"]).fillna(-999)

    # Prefer highest Sherlock reliability per object, then dedupe
    if "classificationReliability" in df_all.columns:
        df_all = (df_all.sort_values("classificationReliability", ascending=False)
                        .drop_duplicates(subset=["objectId"]))
    else:
        df_all = df_all.drop_duplicates(subset=["objectId"])

    # Quick sanity prints
    print("Total unique objects:", len(df_all))
    print("Missing TNS z (tns_z NaN):", int(df_all["tns_z"].isna().sum()))
    print("z_spec == -999:", int((df_all["z_spec"] == -999).sum()))
    print("Missing Sherlock z (sherlock_z NaN):", int(df_all["sherlock_z"].isna().sum()))

else:
    df_all = pd.DataFrame(columns=["objectId"])
    print("Total unique objects: 0")

out_path = "lasair_snii_objects_sherlock_tns.csv"
df_all.to_csv(out_path, index=False)
print(f"Saved: {out_path}")
df_all["z"] = df_all["z_spec"]
df_all.to_csv(out_path, index=False)

Total unique objects: 1785
Missing TNS z (tns_z NaN): 9
z_spec == -999: 9
Missing Sherlock z (sherlock_z NaN): 309
Saved: lasair_snii_objects_sherlock_tns.csv


In [6]:
in_path  = "lasair_snii_objects_sherlock_tns.csv"          
out_path = "lasair_snii_objects_sherlock_tns_complete.csv"                  

# Read CSV and normalize common null-like values to NaN
df = pd.read_csv(
    in_path,
    keep_default_na=True,
    na_values=["", " ", "NA", "N/A", "na", "n/a", "Null", "NULL", "null", "None", "none", "-"]
)

# Strip whitespace from column names
df.columns = df.columns.str.strip()
df = df.drop(columns=["z_best", "z_host", "z_spec", "tns_z", "sherlock_z"])

# NaN report
nan_counts = df.isna().sum().sort_values(ascending=False)
total_rows = len(df)
nan_percent = (nan_counts / total_rows * 100).round(2)

nan_report = (
    pd.DataFrame({
        "column": nan_counts.index,
        "nan_count": nan_counts.values,
        "nan_percent": nan_percent.values
    })
    .query("nan_count > 0")  # only columns with at least one NaN
    .reset_index(drop=True)
)

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {df.isna().any(axis=1).sum()}")
print("\nColumns ranked by NaN count (descending):")
if nan_report.empty:
    print("No NaNs found in any column.")
else:
    with pd.option_context("display.max_rows", None, "display.max_colwidth", 100):
        print(nan_report.to_string(index=False))


# Produce no-NaN version (rows with any NaN dropped)
df_clean = df.dropna(how="any").copy()

# Remove duplicate objectIds
if "objectId" in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=["objectId"])

# Save cleaned data
df_clean.to_csv(out_path, index=False)

print(f"\nInput rows:  {len(df)}")
print(f"Kept rows:   {len(df_clean)} (no NaNs in any column)")
print(f"Wrote file:  {out_path}")


Total rows: 1785
Rows with at least one NaN: 827

Columns ranked by NaN count (descending):
                   column  nan_count  nan_percent
               dmdt_g_err        512        28.68
               dmdt_r_err        384        21.51
                 distance        306        17.14
  physical_separation_kpc        304        17.03
                   dmdt_g        189        10.59
                   sgmag1        128         7.17
                   srmag1        128         7.17
                distpsnr1        128         7.17
                   dmdt_r         44         2.46
classificationReliability         20         1.12
     eastSeparationArcsec         20         1.12
    northSeparationArcsec         20         1.12
         separationArcsec         20         1.12
      catalogue_object_id         20         1.12

Input rows:  1785
Kept rows:   958 (no NaNs in any column)
Wrote file:  lasair_snii_objects_sherlock_tns_complete.csv


## Querying SN Ib/c objects

In [7]:
# Lasair columns to pull
obj_cols = [
    "gmag","rmag","g_minus_r","jd_g_minus_r","jdmin",
    "ramean","decmean","glatmean",
    "dmdt_g","mag_g02","mag_g08","mag_g28",
    "maggmax","maggmean","maggmin",
    "mag_r02","mag_r08","mag_r28",
    "dmdt_r","magrmax","magrmin",
    "distpsnr1","dmdt_g_err","dmdt_r_err","sgmag1","srmag1"
]

sh_cols = [
    "classification","catalogue_object_id","separationArcsec",
    "northSeparationArcsec","eastSeparationArcsec","physical_separation_kpc",
    "distance","classificationReliability","major_axis_arcsec"
]

tns_cols = [
    "tns_name","type","ra","decl"
]

# Build query
selected = (
    "objects.objectId AS objectId, "
    + ", ".join(f"objects.{c}" for c in obj_cols) + ", "
    + ", ".join(f"sherlock_classifications.{c}" for c in sh_cols) + ", "
    + "sherlock_classifications.z AS sherlock_z, "
    + ", ".join(f"crossmatch_tns.{c}" for c in tns_cols) + ", "
    + "crossmatch_tns.z AS tns_z, "
    "IF(ABS(objects.glatmean)<15., 'yes', 'no') AS galactic_plane, "
    "(objects.jd_g_minus_r - objects.jdmin) AS g_r_days_from_fl"
)

tables = "objects, sherlock_classifications, watchlist_hits, crossmatch_tns"

type_like = (
    " (crossmatch_tns.type LIKE 'SN Ib%%' "
    "OR crossmatch_tns.type LIKE 'SN Ic%%' "
    "OR crossmatch_tns.type LIKE 'SN Ib/c%%') "
)
conditions = (
    "objects.objectId = sherlock_classifications.objectId "
    "AND objects.objectId = watchlist_hits.objectId "
    "AND watchlist_hits.wl_id = 1 " 
    "AND watchlist_hits.name = crossmatch_tns.tns_name "
    f"AND {type_like} "
    "AND objects.g_minus_r IS NOT NULL"
)

def fetch_page(limit=1000, offset=0):
    rows = L.query(
        selected=selected,
        tables=tables,
        conditions=conditions,
        limit=limit,
        offset=offset,
    )
    return pd.DataFrame(rows)


# Page through results
BATCH = 10_000
offset = 0
dfs = []
seen = set()

while True:
    df_page = fetch_page(limit=BATCH, offset=offset)
    if df_page.empty:
        break

    # Deduplicate by objectId across pages
    new_mask = ~df_page["objectId"].isin(seen)
    df_new = df_page.loc[new_mask].copy()

    if not df_new.empty:
        dfs.append(df_new)
        seen.update(df_new["objectId"].tolist())

    if len(df_page) < BATCH:
        break

    offset += BATCH


# Post-process and save
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    # Ensure numeric z columns 
    for c in ["major_axis_arcsec"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["major_axis_arcsec"] = df_all["major_axis_arcsec"].fillna(-999)
    
    # Ensure numeric z columns
    for c in ["tns_z", "sherlock_z"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["z_spec"] = df_all["tns_z"].fillna(-999)

    # Keep Sherlock redshift separately
    df_all["z_host"] = df_all["sherlock_z"]

    # "Best available" redshift
    df_all["z_best"] = df_all["tns_z"].combine_first(df_all["sherlock_z"]).fillna(-999)

    # Prefer highest Sherlock reliability per object, then dedupe
    if "classificationReliability" in df_all.columns:
        df_all = (df_all.sort_values("classificationReliability", ascending=False)
                        .drop_duplicates(subset=["objectId"]))
    else:
        df_all = df_all.drop_duplicates(subset=["objectId"])

    # Quick sanity prints
    print("Total unique objects:", len(df_all))
    print("Missing TNS z (tns_z NaN):", int(df_all["tns_z"].isna().sum()))
    print("z_spec == -999:", int((df_all["z_spec"] == -999).sum()))
    print("Missing Sherlock z (sherlock_z NaN):", int(df_all["sherlock_z"].isna().sum()))

else:
    df_all = pd.DataFrame(columns=["objectId"])
    print("Total unique objects: 0")

out_path = "lasair_snibc_objects_sherlock_tns.csv"
df_all.to_csv(out_path, index=False)
print(f"Saved: {out_path}")
df_all["z"] = df_all["z_spec"]
df_all.to_csv(out_path, index=False)

Total unique objects: 526
Missing TNS z (tns_z NaN): 3
z_spec == -999: 3
Missing Sherlock z (sherlock_z NaN): 90
Saved: lasair_snibc_objects_sherlock_tns.csv


In [8]:
in_path  = "lasair_snibc_objects_sherlock_tns.csv"          
out_path = "lasair_snibc_objects_sherlock_tns_complete.csv" 

# Read CSV and normalize common null-like values to NaN
df = pd.read_csv(
    in_path,
    keep_default_na=True,
    na_values=["", " ", "NA", "N/A", "na", "n/a", "Null", "NULL", "null", "None", "none", "-"]
)

# Strip whitespace from column names
df.columns = df.columns.str.strip()
df = df.drop(columns=["z_best", "z_host", "z_spec", "tns_z", "sherlock_z"])

# NaN report
nan_counts = df.isna().sum().sort_values(ascending=False)
total_rows = len(df)
nan_percent = (nan_counts / total_rows * 100).round(2)

nan_report = (
    pd.DataFrame({
        "column": nan_counts.index,
        "nan_count": nan_counts.values,
        "nan_percent": nan_percent.values
    })
    .query("nan_count > 0")  
    .reset_index(drop=True)
)

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {df.isna().any(axis=1).sum()}")
print("\nColumns ranked by NaN count (descending):")
if nan_report.empty:
    print("No NaNs found in any column.")
else:
    with pd.option_context("display.max_rows", None, "display.max_colwidth", 100):
        print(nan_report.to_string(index=False))


# Produce no-NaN version (rows with any NaN dropped)
df_clean = df.dropna(how="any").copy()

# Remove duplicate objectIds just in case
if "objectId" in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=["objectId"])

# Save cleaned data
df_clean.to_csv(out_path, index=False)

print(f"\nInput rows:  {len(df)}")
print(f"Kept rows:   {len(df_clean)} (no NaNs in any column)")
print(f"Wrote file:  {out_path}")

Total rows: 526
Rows with at least one NaN: 258

Columns ranked by NaN count (descending):
                   column  nan_count  nan_percent
               dmdt_g_err        161        30.61
               dmdt_r_err        121        23.00
                 distance         90        17.11
  physical_separation_kpc         90        17.11
                   dmdt_g         61        11.60
                   sgmag1         36         6.84
                   srmag1         36         6.84
                distpsnr1         36         6.84
                   dmdt_r         13         2.47
classificationReliability          4         0.76
     eastSeparationArcsec          4         0.76
    northSeparationArcsec          4         0.76
         separationArcsec          4         0.76
      catalogue_object_id          4         0.76

Input rows:  526
Kept rows:   268 (no NaNs in any column)
Wrote file:  lasair_snibc_objects_sherlock_tns_complete.csv


## Querying Exotic objects

In [9]:
# Columns to pull
obj_cols = [
    "gmag","rmag","g_minus_r","jd_g_minus_r","jdmin",
    "ramean","decmean","glatmean",
    "dmdt_g","mag_g02","mag_g08","mag_g28",
    "maggmax","maggmean","maggmin",
    "mag_r02","mag_r08","mag_r28",
    "dmdt_r","magrmax","magrmin",
    "distpsnr1","dmdt_g_err","dmdt_r_err","sgmag1","srmag1"
]

sh_cols = [
    "classification","catalogue_object_id","separationArcsec",
    "northSeparationArcsec","eastSeparationArcsec","physical_separation_kpc",
    "distance","classificationReliability","major_axis_arcsec"
]

tns_cols = [
    "tns_name","type","ra","decl"
]

# Build query
selected = (
    "objects.objectId AS objectId, "
    + ", ".join(f"objects.{c}" for c in obj_cols) + ", "
    + ", ".join(f"sherlock_classifications.{c}" for c in sh_cols) + ", "
    + "sherlock_classifications.z AS sherlock_z, "
    + ", ".join(f"crossmatch_tns.{c}" for c in tns_cols) + ", "
    + "crossmatch_tns.z AS tns_z, "
    "IF(ABS(objects.glatmean)<15., 'yes', 'no') AS galactic_plane, "
    "(objects.jd_g_minus_r - objects.jdmin) AS g_r_days_from_fl"
)

tables = "objects, sherlock_classifications, watchlist_hits, crossmatch_tns"

type_like = (
    " (crossmatch_tns.type LIKE 'TDE%%' "
    "OR crossmatch_tns.type LIKE 'SLSN-II%%' "
    "OR crossmatch_tns.type LIKE 'Kilonova%%' "
    "OR crossmatch_tns.type LIKE 'AGN%%' "
    "OR crossmatch_tns.type LIKE 'SLSN-I%%') "
)

conditions = (
    "objects.objectId = sherlock_classifications.objectId "
    "AND objects.objectId = watchlist_hits.objectId "
    "AND watchlist_hits.wl_id = 1 " # TNS watchlist
    "AND watchlist_hits.name = crossmatch_tns.tns_name "
    f"AND {type_like} "
    "AND objects.g_minus_r IS NOT NULL"
)


def fetch_page(limit=1000, offset=0):
    rows = L.query(
        selected=selected,
        tables=tables,
        conditions=conditions,
        limit=limit,
        offset=offset,
    )
    return pd.DataFrame(rows)


# Page through results
BATCH = 10_000
offset = 0
dfs = []
seen = set()

while True:
    df_page = fetch_page(limit=BATCH, offset=offset)
    if df_page.empty:
        break

    # Deduplicate by objectId across pages
    new_mask = ~df_page["objectId"].isin(seen)
    df_new = df_page.loc[new_mask].copy()

    if not df_new.empty:
        dfs.append(df_new)
        seen.update(df_new["objectId"].tolist())

    if len(df_page) < BATCH:
        break

    offset += BATCH


# Post-process and save
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    # Ensure numeric z columns 
    for c in ["major_axis_arcsec"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["major_axis_arcsec"] = df_all["major_axis_arcsec"].fillna(-999)
    
    # Ensure numeric z columns 
    for c in ["tns_z", "sherlock_z"]:
        if c in df_all.columns:
            df_all[c] = pd.to_numeric(df_all[c], errors="coerce")
    
    # Spectroscopic redshift feature (TNS only)
    df_all["z_spec"] = df_all["tns_z"].fillna(-999)

    # Keep Sherlock redshift separately (often host/catalogue/photo-z)
    df_all["z_host"] = df_all["sherlock_z"]

    # "Best available" redshift (if you ever want it)
    df_all["z_best"] = df_all["tns_z"].combine_first(df_all["sherlock_z"]).fillna(-999)

    # Prefer highest Sherlock reliability per object, then dedupe
    if "classificationReliability" in df_all.columns:
        df_all = (df_all.sort_values("classificationReliability", ascending=False)
                        .drop_duplicates(subset=["objectId"]))
    else:
        df_all = df_all.drop_duplicates(subset=["objectId"])

    # Quick sanity prints
    print("Total unique objects:", len(df_all))
    print("Missing TNS z (tns_z NaN):", int(df_all["tns_z"].isna().sum()))
    print("z_spec == -999:", int((df_all["z_spec"] == -999).sum()))
    print("Missing Sherlock z (sherlock_z NaN):", int(df_all["sherlock_z"].isna().sum()))

else:
    df_all = pd.DataFrame(columns=["objectId"])
    print("Total unique objects: 0")

out_path = "lasair_exotic_objects_sherlock_tns.csv"
df_all.to_csv(out_path, index=False)
print(f"Saved: {out_path}")
df_all["z"] = df_all["z_spec"]
df_all.to_csv(out_path, index=False)

Total unique objects: 389
Missing TNS z (tns_z NaN): 4
z_spec == -999: 4
Missing Sherlock z (sherlock_z NaN): 107
Saved: lasair_exotic_objects_sherlock_tns.csv


In [10]:
in_path  = "lasair_exotic_objects_sherlock_tns.csv"          
out_path = "lasair_exotic_objects_sherlock_tns_complete.csv" 

# Read CSV and normalize common null-like values to NaN
df = pd.read_csv(
    in_path,
    keep_default_na=True,
    na_values=["", " ", "NA", "N/A", "na", "n/a", "Null", "NULL", "null", "None", "none", "-"]
)

# Strip whitespace from column names
df.columns = df.columns.str.strip()
df = df.drop(columns=["z_best", "z_host", "z_spec", "tns_z", "sherlock_z"])

# NaN report
nan_counts = df.isna().sum().sort_values(ascending=False)
total_rows = len(df)
nan_percent = (nan_counts / total_rows * 100).round(2)

nan_report = (
    pd.DataFrame({
        "column": nan_counts.index,
        "nan_count": nan_counts.values,
        "nan_percent": nan_percent.values
    })
    .query("nan_count > 0")  # only columns with at least one NaN
    .reset_index(drop=True)
)

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {df.isna().any(axis=1).sum()}")
print("\nColumns ranked by NaN count (descending):")
if nan_report.empty:
    print("No NaNs found in any column.")
else:
    with pd.option_context("display.max_rows", None, "display.max_colwidth", 100):
        print(nan_report.to_string(index=False))

# Produce no-NaN version (rows with any NaN dropped)
df_clean = df.dropna(how="any").copy()

# Remove duplicate objectIds just in case
if "objectId" in df_clean.columns:
    before = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=["objectId"])

# Save cleaned data
df_clean.to_csv(out_path, index=False)

print(f"\nInput rows:  {len(df)}")
print(f"Kept rows:   {len(df_clean)} (no NaNs in any column)")
print(f"Wrote file:  {out_path}")

Total rows: 389
Rows with at least one NaN: 192

Columns ranked by NaN count (descending):
                   column  nan_count  nan_percent
                 distance        107        27.51
  physical_separation_kpc        107        27.51
               dmdt_r_err         85        21.85
               dmdt_g_err         84        21.59
      catalogue_object_id         40        10.28
         separationArcsec         40        10.28
    northSeparationArcsec         40        10.28
     eastSeparationArcsec         40        10.28
classificationReliability         40        10.28
                   sgmag1         23         5.91
                   srmag1         23         5.91
                distpsnr1         23         5.91
                   dmdt_r         22         5.66
                   dmdt_g         20         5.14

Input rows:  389
Kept rows:   197 (no NaNs in any column)
Wrote file:  lasair_exotic_objects_sherlock_tns_complete.csv


## Combine all files into one

In [11]:
# Input files 
inputs = [
    ("lasair_snia_objects_sherlock_tns_complete.csv",   "SN Ia"),
    ("lasair_snii_objects_sherlock_tns_complete.csv",   "SN II"),
    ("lasair_snibc_objects_sherlock_tns_complete.csv",  "SN Ib/c"),
    ("lasair_exotic_objects_sherlock_tns_complete.csv", "Exotic"),
]

output_path = "training_data.csv"
dedupe_by_object_id = True  

dfs = []
total_rows = 0

for path, label in inputs:
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    if "objectId" in df.columns:
        df["objectId"] = df["objectId"].astype(str).str.strip()
    # add source label
    df["source_label"] = label
    total_rows += len(df)
    print(f"{label:7s}: read {len(df)} rows from {path}")
    dfs.append(df)

# Concatenate with union of columns
combined = pd.concat(dfs, ignore_index=True, sort=False)

before = len(combined)
if dedupe_by_object_id and "objectId" in combined.columns:
    combined = combined.drop_duplicates(subset=["objectId"])
    print(f"De-duplicated by objectId: {before} → {len(combined)} rows")

combined.to_csv(output_path, index=False)
print(f"\nWrote combined training file: {output_path}")
print(f"Source rows total: {total_rows} | Combined rows: {len(combined)} | Columns: {len(combined.columns)}")

SN Ia  : read 4062 rows from lasair_snia_objects_sherlock_tns_complete.csv
SN II  : read 958 rows from lasair_snii_objects_sherlock_tns_complete.csv
SN Ib/c: read 268 rows from lasair_snibc_objects_sherlock_tns_complete.csv
Exotic : read 197 rows from lasair_exotic_objects_sherlock_tns_complete.csv
De-duplicated by objectId: 5485 → 5479 rows

Wrote combined training file: training_data.csv
Source rows total: 5485 | Combined rows: 5479 | Columns: 44
